In [ ]:
from IPython.core.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
# %matplotlib inline
%matplotlib inline
from bact_analysis_bessyii.bba.calc import load_calib_data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

# Quadrupole calibration factors check

Checks for the calibration factors. Results found

* Similar in size: thus correspond to mid field strength of an rectangular model
* 

Technical data. Source:


* Quadrupole radius: 35 mm " Technischer Bericht BESSY TB 210/97 April 1997"
* Number of windings: Peter's hand note

    * 28 for the main coil
    * 75 for the muxer helper coil


## Load Data

In [ ]:
calib = load_calib_data()

In [ ]:
calib;

## First consistency checks 

Data as retrieved: but these should be absolute values: the difference is quite large . Furthermore it is checked that 

* hw2phys
* phys2hw

are the inverse values as expected.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=[8, 6])
ax.plot(
    np.absolute(calib.hw2phys), '-'    ,
    np.absolute(1./calib.phys2hw), '.'
)
ax.set_xlabel('Magnet')
ax.set_ylabel('hw2phys [T/(m A)]')
[tick.set_rotation('vertical') for tick in ax.get_xmajorticklabels()]
plt.xticks(fontsize='xx-small');

In [ ]:
tick = ax.get_xmajorticklabels()[0]

In [ ]:
tick.set_rotation('vertical')

Relative difference: variation of roughly -6 to  4 % expected

In [ ]:
fig, ax = plt.subplots(1,1, figsize=[8, 6])
qhw2p = np.absolute(calib.hw2phys)
ax.plot((qhw2p/qhw2p.mean()-1)*100)
ax.set_xlabel('Magnet')
ax.set_ylabel('relativ gradient variaton in percent');
[tick.set_rotation('vertical') for tick in ax.get_xmajorticklabels()];
plt.xticks(fontsize='xx-small');

Variation with length. Expected difference for the different length

* 200: +2 - +4 percent
* 250: +1 - -1 precent
* 500: -4 - -6 precent

Assuming uniform steel properties for all magnets one would assume that the longer magnets would show a slightly stronger gradient. 

In [ ]:
calib;

In [ ]:
idx = calib.length.argsort()
length = calib.length[idx.values]
strength = qhw2p.isel(name=idx.values)
fig, ax = plt.subplots(1,1)

ax.plot(length*1000, (strength/strength.mean()-1) *100, 'o')
ax.set_xlabel('length [mm]')
ax.set_ylabel('relativ gradient variaton in percent');

### Cross check with ideal number of laminations

$$
   \frac{G r^2}  {2 \mu_0} = N I
$$

$$
G  = 2 \mu_0\frac{N I}{r^2}
$$

In [ ]:
mu0 = 4 * np.pi * 1e-7

def calculate_gradient(*, r, N, I):
    G = 2 * mu0 * (N * I) / r**2
    return G

In [ ]:

r = 35e-3
Nturns = 28



In [ ]:
calculate_gradient(r=r,N=75,I=5)

Peter's hand note gfives a value of $\approx$ 0.769 T/m for 5 A

### Calculating B rho

Radius is given in the sql table

In [ ]:
rho = 4.354


$$
    \frac {1}{\rho} = \frac{q}{p_0} B_0
$$

$$
    B_0 = \frac{p_0}{q}\frac 1 \rho
$$

In [ ]:
import scipy.constants

In [ ]:
1.7e9 / 511e3

In [ ]:
Brho = scipy.constants.electron_mass * 3327 * scipy.constants.speed_of_light / scipy.constants.elementary_charge
B0 = Brho / rho
B0 = 1.29932
Brho, B0

In [ ]:
Brho = B0 * rho

Gradient per turn and per Amp

In [ ]:
GpN = 2 * mu0 / r**2
Gp = GpN * Nturns
GpN, Gp

In [ ]:
Brho

According to Peter's hand note

In [ ]:
Brho = 5.67044

In [ ]:
1/(qhw2p.sel(name="Q4M1T2R") / (Gp))

In [ ]:
qhw2p.mean() * Brho / Gp

### Checking hardware 2 physics to ideal value

Hardware to physics allows deriving the K value from the excitation current

The ideal value is expected to be larger than the real one as it neglects the magnetic resistance of the return yoke.
In the plot below one can see that all values are larger than the central one. 

It is assumed that the mechanical yoke length was used for the rectangular approximation, thus 
neglecting the "coil's contribution". 


One can see that the value
* vary strongly from family to family
* vary for magnet 1 or magnet 2 (result of sorting)

It is assumed that an average value was derived for the different magnets.

In [ ]:
krel_ideal = qhw2p * (Brho /Gp)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=[16, 6])
ax.plot((krel_ideal))
ax.set_xlabel('Magnet')
ax.set_ylabel('hw2phys devided by value expected from idealistic model');
# [tick.set_rotation('vertical') for tick in ax.get_xmajorticklabels()];

def align(tick):
    tick.set_rotation(45) 
    tick.set_verticalalignment('top')
    tick.set_horizontalalignment('right')
    
[align(tick) for tick in ax.get_xmajorticklabels()];
plt.xticks(fontsize='small');

In [ ]:
tick = ax.get_xmajorticklabels()[0]

In [ ]:
tick.set_horizontalalignment

In [ ]:
fig, ax = plt.subplots(1,1, figsize=[24, 6])
ax.plot(calib.length, krel_ideal, '.')
ax.set_xlabel('Magnet')
ax.set_ylabel('hw2phys devided by value expected from ');
# [tick.set_rotation('vertical') for tick in ax.get_xmajorticklabels()];
# plt.xticks(fontsize='xx-small');

### New heading

### Cross check by database

There is a second database providing data for different magnets. This sql based database is crosschecked to the data provided by the mongo database

In [ ]:
import bact2_bessyii.magnets

In [ ]:
df =  bact2_bessyii.magnets.quadrupole_calibration_factors().set_index("name")
df.head()

In [ ]:
calib_sql = xr.Dataset.from_dataframe(df)
calib_sql;

The parameter "al" seems to correspond to the length

In [ ]:
fig, ax = plt.subplots(1, 1, )
ax.plot(calib_sql.al)

In [ ]:
tf = calib_sql.t / calib_sql.al

Division yields that the transfer function is pretty uniform

In [ ]:
fig, ax = plt.subplots(1, 1, )
ax.plot(tf/tf.mean())

Comparison of the "central" transfer function

In [ ]:
fig, ax = plt.subplots(1, 1, )
ax.plot(tf, qhw2p * Brho, '*')
ax.set_xlabel(r'r$t_f$ [T/M A]')
ax.set_ylabel(r'r$k_1$ * B $\rho$ [T/M A]')

Comparison of the transfer function ratio for the different magnets

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.plot((tf / (qhw2p * Brho) -1) * 100, '*')
ax.set_xlabel('magnet')
ax.set_ylabel(r'$\frac{t_f}{rk_1 \cdot B \rho}$ [%]')
[tick.set_rotation('vertical') for tick in ax.get_xmajorticklabels()];
#plt.xticks(fontsize='xx-small');
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(calib_sql.al, (tf / (qhw2p * Brho) -1) * 100, '*')
ax.set_xlabel('l [mm]')
ax.set_ylabel(r'$\frac{t_f}{rk_1 \cdot B \rho}$ [%]')
[tick.set_rotation('vertical') for tick in ax.get_xmajorticklabels()];
#plt.xticks(fontsize='xx-small');
fig.tight_layout()

In [ ]:
def corresponding_length(length):
    l200 = (length>0.23) & (length<0.235)
    l250 = (length>0.27) & (length<0.275)
    l500 = (length>0.5 ) & (length<0.51)
    
    check = l200 + l250 + l500
    assert((check == 1).all())
    
    nl = xr.DataArray(dims=length.dims, coords=length.coords)
    nl.loc[dict(name=l200)] = 0.2
    nl.loc[dict(name=l250)] = 0.25
    nl.loc[dict(name=l500)] = 0.5
    return nl

In [ ]:
mechanical_length = corresponding_length(calib_sql.al)

In [ ]:
set(list(calib_sql.al.values))

In [ ]:
tfm = calib_sql.t / mechanical_length

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(calib_sql.al, (tfm / (qhw2p * Brho) -1) * 100, '*')
ax.set_xlabel('l [mm]')
ax.set_ylabel(r'$\frac{t_f}{rk_1 \cdot B \rho}$ [%]')
#[tick.set_rotation('vertical') for tick in ax.get_xmajorticklabels()];
#plt.xticks(fontsize='xx-small');
fig.tight_layout()

## AT Measurement data 

In [ ]:
 txt = """(QUAD) Fit parameter:   1, Name : Q1M2D1R, value:  2.434852  1/m^2
   (QUAD) Fit parameter:   2, Name : Q1M1T1R, value:  2.432500  1/m^2
   (QUAD) Fit parameter:   3, Name : Q1M2T1R, value:  2.438276  1/m^2
   (QUAD) Fit parameter:   4, Name : Q1M1D2R, value:  2.428871  1/m^2
   (QUAD) Fit parameter:   5, Name : Q1M2D2R, value:  2.430467  1/m^2
   (QUAD) Fit parameter:   6, Name : Q1M1T2R, value:  2.434239  1/m^2
   (QUAD) Fit parameter:   7, Name : Q1M2T2R, value:  2.428813  1/m^2
   (QUAD) Fit parameter:   8, Name : Q1M1D3R, value:  2.432423  1/m^2
   (QUAD) Fit parameter:   9, Name : Q1M2D3R, value:  2.444430  1/m^2
   (QUAD) Fit parameter:  10, Name : Q1M1T3R, value:  2.444282  1/m^2
   (QUAD) Fit parameter:  11, Name : Q1M2T3R, value:  2.437315  1/m^2
   (QUAD) Fit parameter:  12, Name : Q1M1D4R, value:  2.431714  1/m^2
   (QUAD) Fit parameter:  13, Name : Q1M2D4R, value:  2.426608  1/m^2
   (QUAD) Fit parameter:  14, Name : Q1M1T4R, value:  2.434197  1/m^2
   (QUAD) Fit parameter:  15, Name : Q1M2T4R, value:  2.430148  1/m^2
   (QUAD) Fit parameter:  16, Name : Q1M1D5R, value:  2.437458  1/m^2
   (QUAD) Fit parameter:  17, Name : Q1M2D5R, value:  2.433468  1/m^2
   (QUAD) Fit parameter:  18, Name : Q1M1T5R, value:  2.439687  1/m^2
   (QUAD) Fit parameter:  19, Name : Q1M2T5R, value:  2.431409  1/m^2
   (QUAD) Fit parameter:  20, Name : Q1M1D6R, value:  2.432905  1/m^2
   (QUAD) Fit parameter:  21, Name : Q1M2D6R, value:  2.446809  1/m^2
   (QUAD) Fit parameter:  22, Name : Q1M1T6R, value:  2.434695  1/m^2
   (QUAD) Fit parameter:  23, Name : Q1M2T6R, value:  2.425819  1/m^2
   (QUAD) Fit parameter:  24, Name : Q1M1D7R, value:  2.438531  1/m^2
   (QUAD) Fit parameter:  25, Name : Q1M2D7R, value:  2.428425  1/m^2
   (QUAD) Fit parameter:  26, Name : Q1M1T7R, value:  2.440111  1/m^2
   (QUAD) Fit parameter:  27, Name : Q1M2T7R, value:  2.435483  1/m^2
   (QUAD) Fit parameter:  28, Name : Q1M1D8R, value:  2.439362  1/m^2
   (QUAD) Fit parameter:  29, Name : Q1M2D8R, value:  2.429829  1/m^2
   (QUAD) Fit parameter:  30, Name : Q1M1T8R, value:  2.438577  1/m^2
   (QUAD) Fit parameter:  31, Name : Q1M2T8R, value:  2.436092  1/m^2
   (QUAD) Fit parameter:  32, Name : Q1M1D1R, value:  2.438516  1/m^2
   (QUAD) Fit parameter:  33, Name : Q2M2D1R, value: -1.845751  1/m^2
   (QUAD) Fit parameter:  34, Name : Q2M1T1R, value: -1.882292  1/m^2
   (QUAD) Fit parameter:  35, Name : Q2M2T1R, value: -1.842973  1/m^2
   (QUAD) Fit parameter:  36, Name : Q2M1D2R, value: -1.887146  1/m^2
   (QUAD) Fit parameter:  37, Name : Q2M2D2R, value: -1.882148  1/m^2
   (QUAD) Fit parameter:  38, Name : Q2M1T2R, value: -1.825845  1/m^2
   (QUAD) Fit parameter:  39, Name : Q2M2T2R, value: -1.828352  1/m^2
   (QUAD) Fit parameter:  40, Name : Q2M1D3R, value: -1.871478  1/m^2
   (QUAD) Fit parameter:  41, Name : Q2M2D3R, value: -1.868630  1/m^2
   (QUAD) Fit parameter:  42, Name : Q2M1T3R, value: -1.819808  1/m^2
   (QUAD) Fit parameter:  43, Name : Q2M2T3R, value: -1.821105  1/m^2
   (QUAD) Fit parameter:  44, Name : Q2M1D4R, value: -1.881955  1/m^2
   (QUAD) Fit parameter:  45, Name : Q2M2D4R, value: -1.878322  1/m^2
   (QUAD) Fit parameter:  46, Name : Q2M1T4R, value: -1.816713  1/m^2
   (QUAD) Fit parameter:  47, Name : Q2M2T4R, value: -1.839283  1/m^2
   (QUAD) Fit parameter:  48, Name : Q2M1D5R, value: -1.862095  1/m^2
   (QUAD) Fit parameter:  49, Name : Q2M2D5R, value: -1.883766  1/m^2
   (QUAD) Fit parameter:  50, Name : Q2M1T5R, value: -1.817666  1/m^2
   (QUAD) Fit parameter:  51, Name : Q2M2T5R, value: -1.837172  1/m^2
   (QUAD) Fit parameter:  52, Name : Q2M1D6R, value: -1.869564  1/m^2
   (QUAD) Fit parameter:  53, Name : Q2M2D6R, value: -1.869799  1/m^2
   (QUAD) Fit parameter:  54, Name : Q2M1T6R, value: -1.864825  1/m^2
   (QUAD) Fit parameter:  55, Name : Q2M2T6R, value: -1.853606  1/m^2
   (QUAD) Fit parameter:  56, Name : Q2M1D7R, value: -1.864121  1/m^2
   (QUAD) Fit parameter:  57, Name : Q2M2D7R, value: -1.884701  1/m^2
   (QUAD) Fit parameter:  58, Name : Q2M1T7R, value: -1.822435  1/m^2
   (QUAD) Fit parameter:  59, Name : Q2M2T7R, value: -1.851085  1/m^2
   (QUAD) Fit parameter:  60, Name : Q2M1D8R, value: -1.870231  1/m^2
   (QUAD) Fit parameter:  61, Name : Q2M2D8R, value: -1.878520  1/m^2
   (QUAD) Fit parameter:  62, Name : Q2M1T8R, value: -1.838834  1/m^2
   (QUAD) Fit parameter:  63, Name : Q2M2T8R, value: -1.866194  1/m^2
   (QUAD) Fit parameter:  64, Name : Q2M1D1R, value: -1.838139  1/m^2
   (QUAD) Fit parameter:  65, Name : Q3M2D1R, value: -2.018753  1/m^2
   (QUAD) Fit parameter:  66, Name : Q3M1T1R, value: -2.513644  1/m^2
   (QUAD) Fit parameter:  67, Name : Q3M2T1R, value: -2.489925  1/m^2
   (QUAD) Fit parameter:  68, Name : Q3M1D2R, value: -2.103146  1/m^2
   (QUAD) Fit parameter:  69, Name : Q3M2D2R, value: -2.113701  1/m^2
   (QUAD) Fit parameter:  70, Name : Q3M1T2R, value: -2.457846  1/m^2
   (QUAD) Fit parameter:  71, Name : Q3M2T2R, value: -2.455187  1/m^2
   (QUAD) Fit parameter:  72, Name : Q3M1D3R, value: -2.113817  1/m^2
   (QUAD) Fit parameter:  73, Name : Q3M2D3R, value: -2.114554  1/m^2
   (QUAD) Fit parameter:  74, Name : Q3M1T3R, value: -2.457973  1/m^2
   (QUAD) Fit parameter:  75, Name : Q3M2T3R, value: -2.459593  1/m^2
   (QUAD) Fit parameter:  76, Name : Q3M1D4R, value: -2.114393  1/m^2
   (QUAD) Fit parameter:  77, Name : Q3M2D4R, value: -2.110309  1/m^2
   (QUAD) Fit parameter:  78, Name : Q3M1T4R, value: -2.454027  1/m^2
   (QUAD) Fit parameter:  79, Name : Q3M2T4R, value: -2.447107  1/m^2
   (QUAD) Fit parameter:  80, Name : Q3M1D5R, value: -2.111989  1/m^2
   (QUAD) Fit parameter:  81, Name : Q3M2D5R, value: -2.111254  1/m^2
   (QUAD) Fit parameter:  82, Name : Q3M1T5R, value: -2.459242  1/m^2
   (QUAD) Fit parameter:  83, Name : Q3M2T5R, value: -2.458475  1/m^2
   (QUAD) Fit parameter:  84, Name : Q3M1D6R, value: -2.093306  1/m^2
   (QUAD) Fit parameter:  85, Name : Q3M2D6R, value: -2.102154  1/m^2
   (QUAD) Fit parameter:  86, Name : Q3M1T6R, value: -2.669755  1/m^2
   (QUAD) Fit parameter:  87, Name : Q3M2T6R, value: -2.296469  1/m^2
   (QUAD) Fit parameter:  88, Name : Q3M1D7R, value: -2.113762  1/m^2
   (QUAD) Fit parameter:  89, Name : Q3M2D7R, value: -2.099826  1/m^2
   (QUAD) Fit parameter:  90, Name : Q3M1T7R, value: -2.469869  1/m^2
   (QUAD) Fit parameter:  91, Name : Q3M2T7R, value: -2.469500  1/m^2
   (QUAD) Fit parameter:  92, Name : Q3M1D8R, value: -2.119245  1/m^2
   (QUAD) Fit parameter:  93, Name : Q3M2D8R, value: -2.103575  1/m^2
   (QUAD) Fit parameter:  94, Name : Q3M1T8R, value: -2.476209  1/m^2
   (QUAD) Fit parameter:  95, Name : Q3M2T8R, value: -2.505566  1/m^2
   (QUAD) Fit parameter:  96, Name : Q3M1D1R, value: -2.030975  1/m^2
   (QUAD) Fit parameter:  97, Name : Q4M2D1R, value:  1.403441  1/m^2
   (QUAD) Fit parameter:  98, Name : Q4M1T1R, value:  2.637748  1/m^2
   (QUAD) Fit parameter:  99, Name : Q4M2T1R, value:  2.574446  1/m^2
   (QUAD) Fit parameter: 100, Name : Q4M1D2R, value:  1.480546  1/m^2
   (QUAD) Fit parameter: 101, Name : Q4M2D2R, value:  1.481529  1/m^2
   (QUAD) Fit parameter: 102, Name : Q4M1T2R, value:  2.583164  1/m^2
   (QUAD) Fit parameter: 103, Name : Q4M2T2R, value:  2.585376  1/m^2
   (QUAD) Fit parameter: 104, Name : Q4M1D3R, value:  1.481568  1/m^2
   (QUAD) Fit parameter: 105, Name : Q4M2D3R, value:  1.485072  1/m^2
   (QUAD) Fit parameter: 106, Name : Q4M1T3R, value:  2.578520  1/m^2
   (QUAD) Fit parameter: 107, Name : Q4M2T3R, value:  2.582798  1/m^2
   (QUAD) Fit parameter: 108, Name : Q4M1D4R, value:  1.499967  1/m^2
   (QUAD) Fit parameter: 109, Name : Q4M2D4R, value:  1.484875  1/m^2
   (QUAD) Fit parameter: 110, Name : Q4M1T4R, value:  2.582721  1/m^2
   (QUAD) Fit parameter: 111, Name : Q4M2T4R, value:  2.581386  1/m^2
   (QUAD) Fit parameter: 112, Name : Q4M1D5R, value:  1.483486  1/m^2
   (QUAD) Fit parameter: 113, Name : Q4M2D5R, value:  1.479909  1/m^2
   (QUAD) Fit parameter: 114, Name : Q4M1T5R, value:  2.582541  1/m^2
   (QUAD) Fit parameter: 115, Name : Q4M2T5R, value:  2.581872  1/m^2
   (QUAD) Fit parameter: 116, Name : Q4M1D6R, value:  1.484540  1/m^2
   (QUAD) Fit parameter: 117, Name : Q4M2D6R, value:  1.485975  1/m^2
   (QUAD) Fit parameter: 118, Name : Q4M1T6R, value:  2.248588  1/m^2
   (QUAD) Fit parameter: 119, Name : Q4M2T6R, value:  2.564476  1/m^2
   (QUAD) Fit parameter: 120, Name : Q4M1D7R, value:  1.475396  1/m^2
   (QUAD) Fit parameter: 121, Name : Q4M2D7R, value:  1.490089  1/m^2
   (QUAD) Fit parameter: 122, Name : Q4M1T7R, value:  2.589398  1/m^2
   (QUAD) Fit parameter: 123, Name : Q4M2T7R, value:  2.592736  1/m^2
   (QUAD) Fit parameter: 124, Name : Q4M1D8R, value:  1.490483  1/m^2
   (QUAD) Fit parameter: 125, Name : Q4M2D8R, value:  1.481112  1/m^2
   (QUAD) Fit parameter: 126, Name : Q4M1T8R, value:  2.568419  1/m^2
   (QUAD) Fit parameter: 127, Name : Q4M2T8R, value:  2.643976  1/m^2
   (QUAD) Fit parameter: 128, Name : Q4M1D1R, value:  1.406749  1/m^2
   (QUAD) Fit parameter: 129, Name : Q5M1T1R, value: -2.522543  1/m^2
   (QUAD) Fit parameter: 130, Name : Q5M2T1R, value: -2.518521  1/m^2
   (QUAD) Fit parameter: 131, Name : Q5M1T2R, value: -2.582140  1/m^2
   (QUAD) Fit parameter: 132, Name : Q5M2T2R, value: -2.605689  1/m^2
   (QUAD) Fit parameter: 133, Name : Q5M1T3R, value: -2.605526  1/m^2
   (QUAD) Fit parameter: 134, Name : Q5M2T3R, value: -2.595408  1/m^2
   (QUAD) Fit parameter: 135, Name : Q5M1T4R, value: -2.586966  1/m^2
   (QUAD) Fit parameter: 136, Name : Q5M2T4R, value: -2.593508  1/m^2
   (QUAD) Fit parameter: 137, Name : Q5M1T5R, value: -2.599672  1/m^2
   (QUAD) Fit parameter: 138, Name : Q5M2T5R, value: -2.573553  1/m^2
   (QUAD) Fit parameter: 139, Name : Q5M1T6R, value: -1.039136  1/m^2
   (QUAD) Fit parameter: 140, Name : Q5M2T6R, value: -2.508676  1/m^2
   (QUAD) Fit parameter: 141, Name : Q5M1T7R, value: -2.613940  1/m^2
   (QUAD) Fit parameter: 142, Name : Q5M2T7R, value: -2.587546  1/m^2
   (QUAD) Fit parameter: 143, Name : Q5M1T8R, value: -2.511650  1/m^2
   (QUAD) Fit parameter: 144, Name : Q5M2T8R, value: -2.519138  1/m^2
   (QUAD) Fit parameter: 145, Name : QIT6R , value: -1.078482  1/m^2
"""
lines = txt.split('\n')

In [ ]:
lines[-1]

In [ ]:
import re

In [ ]:
cols = lines[1].split(",")

In [ ]:
cols

In [ ]:
cols[0]

In [ ]:
match = re.match("\s*\((\S*)\)\s", cols[0])

In [ ]:
match.groups()

In [ ]:
match = re.match("\s*Name\s*:\s*(\S*)\s*", cols[1])
match.groups()

In [ ]:
match= re.match(" value:\s*(\S*)\s* 1/m\^2", cols[2])
match.groups()

In [ ]:
match_fit_type = re.compile("\s*\((\S*)\).*:\s*(\d*)")
match_element_name = re.compile("\s*Name\s*:\s*(\S*)\s*")
match_value = re.compile(" value:\s*(\S*)\s* 1/m\^2")

In [ ]:
def convert_line(line):
    cols = line.split(",")
    fit_txt, elem_txt, val_txt = cols
    
    try:
        fitm = match_fit_type.match(fit_txt)
        family, nelem = fitm.groups()
    except Exception:
        print("Failed to convert line", line)
        raise

    nelem = int(nelem)
    
    namem = match_element_name.match(elem_txt)
    name, = namem.groups()
    
    valm = match_value.match(val_txt)    
    val, = valm.groups()
    val = float(val)
    
    return dict(family=family, nelem=nelem, name=name, val=val)

entries = [convert_line(line) for line in lines[:-1]]

In [ ]:
entries;

In [ ]:
df = pd.DataFrame(entries).infer_objects()
df

In [ ]:
df.to_json('bessy2_quad_strength_measured_loco.json', indent=4)

In [ ]:
import thor_scsi.lib as ts_lib

In [ ]:
from thor_scsi.utils import accelerator
from thor_scsi.factory import accelerator_from_config

In [ ]:
pd.read_json('bessy2_quad_strength_measured_loco.json')

In [ ]:
import os.path

In [ ]:
lattice_dir = os.path.join(
    os.environ["HOME"], "cpp",  "dt4acc", 
    # "thor-scsi-lib", "python", "tests", 
    "lattices"
)
lattice_dir

In [ ]:
os.listdir(lattice_dir)

In [ ]:
lat_file = os.path.join(lattice_dir, 'b2_stduser_beamports_blm_tracy_corr.lat')
# lat_file = os.path.join(lattice_dir, '')


In [ ]:
acc = accelerator_from_config(lat_file)

In [ ]:
df_lu = df.copy()
df_lu.loc[:, "name"] = [tmp.lower() for tmp in df.name]
df_lu = df_lu.set_index("name")
df_lu

In [ ]:
d = {elem.name: elem for elem in acc}

In [ ]:
tmp = d['q1m1t1r']
tmp.getMainMultipoleStrengthComponent()

In [ ]:
df_lu2 = df_lu.reindex(columns=list(df_lu.columns) + ['model', 'rel'])
for t_name in df_lu.index:
    if t_name == 'qit6r':
        continue
    df_lu2.loc[t_name, "model"] = d[t_name].getMainMultipoleStrengthComponent()
df_lu2.loc[:, "rel"] = (df_lu2.model / df_lu.val - 1) * 1e4
df_lu2

In [ ]:
fig, ax = plt.subplots(1, 1 , figsize=[16, 12])
df_lu2.rel.plot()

In [ ]:
### Put the strength into the model

In [ ]:
for name, val in df_lu.val.items():
    if name == "qit6r":
        # This one is split 
        acc.find(name + 'l', 0).setMainMultipoleStrength(val)
        acc.find(name + 'r', 0).setMainMultipoleStrength(val)
    else:
        acc.find(name, 0).setMainMultipoleStrength(val)
    # print(val)
    # break

In [ ]:
from thor_scsi.utils.linear_optics import compute_Twiss_along_lattice
import gtpsa
desc = gtpsa.desc(6, 2)
compute_Twiss_along_lattice?

In [ ]:
from bact_analysis.utils.preprocess import rename_doublicates

In [ ]:
ttwiss2 = compute_Twiss_along_lattice(2, acc, desc=desc, mapping=gtpsa.default_mapping())
ttwiss2

In [ ]:
_, pos_names_without_doublets = rename_doublicates(ttwiss2.names.values.tolist())

In [ ]:
ttwiss = (
    xr.Dataset(dict(beta=ttwiss2.twiss.sel(par="beta")))
    .assign(dict(dmu=ttwiss2.twiss.sel(par="dnu"), ds=ttwiss2.s))
)

ttwiss = ttwiss.assign(mu=ttwiss.dmu.cumsum(dim="index")).rename(index="pos").assign_coords(dict(pos=pos_names_without_doublets))
ttwiss

In [ ]:
ls

In [ ]:
ttwiss.to_netcdf('bessyii_twiss_thor_scsi_twin.nc')

In [ ]:
os.getcwd()

In [ ]:
ls -l

In [ ]:
plt.plot(
    ttwiss.mu.sel(plane='x'), '-',
    ttwiss.dmu.sel(plane='x').cumsum(), '-'
        )

In [ ]:
plt.plot(
    ttwiss.mu.sel(plane='y'), '-',
    ttwiss.dmu.sel(plane='y').cumsum(), '-'
)